In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
# load source dataset containing point coordinates
df = gpd.read_file('raw/jugendfreizeiteinrichtungen.geojson')

# we want to find the respective plr for each point coordinate, so we need the plr polygons
plr = gpd.read_file('../plr/plr_only.geojson')

# plr without geometry (for merging later on)
plr_no_geo = plr.drop(columns='geometry').sort_values(by='PLR_ID')

# check for same crs (coordinate reference system)
df.crs == plr.crs

True

In [3]:
# Spatial join assigns each point to its respective plr
df_with_plr = gpd.sjoin(df, plr, how='inner', predicate='within')

print("df shape:", df.shape, "\ndf_with_plr shape:", df_with_plr.shape)

df shape: (407, 21) 
df_with_plr shape: (407, 24)


In [4]:
df_with_plr.head()

,e_nr,e_name,e_bez_nr,e_bez,e_plz,e_ort,e_strasse,e_hnr,e_zus,e_tel,...,zielgr2,gend1,gend2,t_name,t_art,t_web,geometry,index_right,PLR_ID,PLR_NAME
0,01010003,Freizeithaus am Mauerpark,01,Mitte,10435,Berlin,Schwedter Str.,234,NaN,030/ 4490741,...,NaN,NaN,NaN,BA Mitte von Berlin,öffentlicher Träger,NaN,POINT (391841.900 5822078.300),275,01100416,Arkonaplatz
1,01010004,Kinder- & Jugendzentrum Die OASE,01,Mitte,10179,Berlin,Wallstr.,43,NaN,030 24049645,...,NaN,NaN,NaN,Bezirksamt Mitte von Berlin,öffentlicher Träger,https://www.berlin.de/ba-mitte/,POINT (392267.100 5819268.400),137,01100312,Heine-Viertel West
2,01010016,Jugendkulturzentrum Weinmeisterhaus,01,Mitte,10178,Berlin,Weinmeisterstraße,15,NaN,030 270044971,...,NaN,NaN,NaN,WeTeK Berlin gGmbH,freier Träger,https://www.wetek.de,POINT (391793.300 5820695.800),332,01100309,Oranienburger Straße
10,01010047,RAA Schüler*innenclub am JGMM,01,Mitte,10115,Berlin,Große Hamburger Str.,27,NaN,0170-7617578,...,NaN,NaN,NaN,RAA Berlin,freier Träger,https://raa-berlin.de/,POINT (391408.600 5820634.100),332,01100309,Oranienburger Straße
3,01010017,Jugendclub Ikarus,01,Mitte,10117,Berlin,Wilhelmstraße,51,NaN,030/2292018,...,NaN,NaN,NaN,technische Jugendfreizeit- und Bildungseselsch...,freier Träger,https://www.tjfbg.de/,POINT (390271.000 5819284.500),457,01100205,Wilhelmstraße


In [5]:
df_with_plr.columns

Index(['e_nr', 'e_name', 'e_bez_nr', 'e_bez', 'e_plz', 'e_ort', 'e_strasse',
       'e_hnr', 'e_zus', 'e_tel', 'e_web', 'e_platz', 'e_art', 'zielgr1',
       'zielgr2', 'gend1', 'gend2', 't_name', 't_art', 't_web', 'geometry',
       'index_right', 'PLR_ID', 'PLR_NAME'],
      dtype='object')

In [6]:
# only keep summary relevant features, get rid of geometry -> regular pandas dataframe
feature_list = ['PLR_ID', 'e_name', 'e_art', 't_art']
df_with_plr = df_with_plr[feature_list]
df_with_plr.head()

,PLR_ID,e_name,e_art,t_art
0,01100416,Freizeithaus am Mauerpark,Jugendfreizeiteinrichtungen (JFE),öffentlicher Träger
1,01100312,Kinder- & Jugendzentrum Die OASE,Jugendfreizeiteinrichtungen (JFE),öffentlicher Träger
2,01100309,Jugendkulturzentrum Weinmeisterhaus,Jugendfreizeiteinrichtungen (JFE),freier Träger
10,01100309,RAA Schüler*innenclub am JGMM,Jugendfreizeiteinrichtungen (JFE),freier Träger
3,01100205,Jugendclub Ikarus,Jugendfreizeiteinrichtungen (JFE),freier Träger


In [7]:
# Summarize by groupby
gb = df_with_plr.groupby('PLR_ID')

count = gb.size().reset_index(name='jfe_count')
names = gb['e_name'].apply(list).reset_index(name='jfe_names')
art = gb['e_art'].value_counts().unstack().reset_index().fillna(0)
traeger = gb['t_art'].value_counts().unstack().reset_index().fillna(0)

In [8]:
# Merge
result = plr_no_geo \
    .merge(count, on='PLR_ID', how='outer') \
    .merge(names, on='PLR_ID', how='outer') \
    .merge(art, on='PLR_ID', how='outer') \
    .merge(traeger, on='PLR_ID', how='outer')

result.shape

(542, 8)

In [9]:
result.head()

,PLR_ID,PLR_NAME,jfe_count,jfe_names,Jugendfreizeiteinrichtungen (JFE),PbS oder ASP oder Kinderfarm,freier Träger,öffentlicher Träger
0,01100101,Stülerstraße,NaN,NaN,NaN,NaN,NaN,NaN
1,01100102,Großer Tiergarten,NaN,NaN,NaN,NaN,NaN,NaN
2,01100103,Lützowstraße,2.0,"[Kindertreff Villa Lützow, JFE Queer]",2.0,0.0,2.0,0.0
3,01100104,Körnerstraße,1.0,[Jugendbögen Pohlstraße],1.0,0.0,1.0,0.0
4,01100205,Wilhelmstraße,1.0,[Jugendclub Ikarus],1.0,0.0,1.0,0.0


In [10]:
result.to_csv("jugendfreizeiteinrichtungen_per_plr.csv", index=False)